In [1]:
#import libraries
import pandas as pd
import os
import zipfile
import shutil

#import torch libraries

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# clone the repo
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 14.94 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
# extract the content
extract_dir = "/content/Train"
os.makedirs(extract_dir, exist_ok=True)

#open and extract the zip file
for i in range(1,3):
  with zipfile.ZipFile(f"/content/Qualcomm-DL-Hackathon/train/images part-{i}.zip","r") as zip_ref:
    zip_ref.extractall(extract_dir)

In [4]:
train_data_csv = pd.read_csv("/content/Qualcomm-DL-Hackathon/train/train.csv")
test = pd.read_csv("/content/Qualcomm-DL-Hackathon/test.csv")

In [5]:
# extract images and split them into train and test

destination_folder = "/content/Train/train_images"

if not os.path.exists(destination_folder):
  os.makedirs(destination_folder)

for i in range(1,3):
  source_folder = f"/content/Train/images part-{i}"
  for root, dirs, files in os.walk(source_folder):
    for file in files:
      src_file = os.path.join(root, file)
      shutil.copy2(src_file, destination_folder)



In [6]:
#splitting datasets into train and test
test_images = list(test["image_names"])
source_folder = f"/content/Train/train_images"
destination_folder = "/content/Train/final_test_images"

if not os.path.exists(destination_folder):
  os.makedirs(destination_folder)
count = 0
for root, dirs, files in os.walk(source_folder):
  for file in files:
    count += 1
    if file in test_images:
      src_file = os.path.join(root, file)
      shutil.move(src_file, destination_folder)


In [7]:
# initialize the res dict to construct final csv at the end
count = 0
res = {"image_names":[], "emergency_or_not":[]}
for root, dirs, files in os.walk(destination_folder):
  for file in files:
    # if count < 1:
    #   print(file, type(file), test_images[0],type(test_images[0]))
    count += 1
    res["image_names"].append(file)


In [8]:
# Creating classes to handle image to label mapping with(for train) and without labels(for test)
from torch.utils.data import Dataset
from PIL import Image

class CustomImageDatasetWithLabels(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.labels_df.iloc[idx, 0])
        image = Image.open(img_name)
        label = int(self.labels_df.iloc[idx, 1])  # Convert label to int
        if self.transform:
            image = self.transform(image)
        return image, label

class CustomImageDatasetWithoutLabels(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_names = os.listdir(image_folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_names[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image

In [9]:
# Tried different combinations with color saturation, vertical flip, etc.
# Found this combination to be better than others
# Here, we resize the image to 256x256 along with horizontal flip and rotation of 15 degrees.
train_transforms = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
])


In [15]:

# Paths to your CSV files and image folders
train_csv = '/content/Qualcomm-DL-Hackathon/train/train.csv'
train_image_folder = '/content/Train/train_images'

# Initialize your datasets
train_dataset = CustomImageDatasetWithLabels(csv_file=train_csv, image_folder=train_image_folder, transform=train_transforms)
test_dataset = CustomImageDatasetWithoutLabels(image_folder=destination_folder, transform=test_transforms)


In [11]:
# Defining a function to split data into train and test sets
def split_data(data,ratio=0.8):
    train_size = int(ratio * len(data))
    test_size = len(data) - train_size
    train_data, test_data = random_split(data, [train_size, test_size])
    return train_data, test_data

In [12]:
# Defining a function to define data loaders
# Found batch size 128 to be the optimal value based on testing
def create_dataloaders(train_data, test_data,b_train=128,b_test=32):
    train_loader = DataLoader(train_data, batch_size=b_train, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=b_test, shuffle=False)
    return train_loader, test_loader


In [16]:
# define the train and test sets
train_loader, test_loader = create_dataloaders(train_dataset, test_dataset)

In [17]:
# Initializing the model
from torchvision.models import resnet50
model = resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,2)

model = model.to(device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 200MB/s]


In [18]:
# Define the optimizer and scehduler
from torch.optim.lr_scheduler import StepLR
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)


In [19]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
    scheduler.step()

Epoch 1/50, Loss: 0.3271, Accuracy: 87.48%
Epoch 2/50, Loss: 0.1504, Accuracy: 93.68%
Epoch 3/50, Loss: 0.1194, Accuracy: 95.57%
Epoch 4/50, Loss: 0.0731, Accuracy: 97.69%
Epoch 5/50, Loss: 0.0589, Accuracy: 98.12%
Epoch 6/50, Loss: 0.0535, Accuracy: 98.18%
Epoch 7/50, Loss: 0.0635, Accuracy: 97.63%
Epoch 8/50, Loss: 0.0461, Accuracy: 98.42%
Epoch 9/50, Loss: 0.0353, Accuracy: 98.72%
Epoch 10/50, Loss: 0.0274, Accuracy: 99.15%
Epoch 11/50, Loss: 0.0205, Accuracy: 99.45%
Epoch 12/50, Loss: 0.0137, Accuracy: 99.64%
Epoch 13/50, Loss: 0.0093, Accuracy: 99.88%
Epoch 14/50, Loss: 0.0106, Accuracy: 99.57%
Epoch 15/50, Loss: 0.0111, Accuracy: 99.51%
Epoch 16/50, Loss: 0.0077, Accuracy: 99.70%
Epoch 17/50, Loss: 0.0073, Accuracy: 99.70%
Epoch 18/50, Loss: 0.0059, Accuracy: 99.82%
Epoch 19/50, Loss: 0.0072, Accuracy: 99.76%
Epoch 20/50, Loss: 0.0051, Accuracy: 99.64%
Epoch 21/50, Loss: 0.0057, Accuracy: 99.70%
Epoch 22/50, Loss: 0.0050, Accuracy: 99.82%
Epoch 23/50, Loss: 0.0045, Accuracy: 99.9

In [ ]:
# Used to evaluate test data if data is split into train and test
def evaluate_test(test_loader):
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_accuracy = 100 * test_correct / test_total
    print(f"Test Accuracy: {test_accuracy:.2f}%")

In [20]:
# Define a function to predict the outputs for test data
results = {}
def predict_op():
  global res
  global results
  res["emergency_or_not"].clear()
  model.eval()
  test_correct = 0
  test_total = 0
  with torch.no_grad():
      for inputs in test_loader:
          inputs = inputs.to(device)

          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          # print([t.item() for t in list(predicted)])
          res["emergency_or_not"].extend([t.item() for t in list(predicted)])
  results = {}
  for i in range(len(res["image_names"])):
    results[res["image_names"][i]] = res["emergency_or_not"][i]


In [21]:
# Define a function to create csv
def create_csv(name):
  temp = []
  import pandas as pd
  data = pd.read_csv("/content/Qualcomm-DL-Hackathon/test.csv")
  for i in list(data["image_names"]):
    temp.append(results[i])
  data["emergency_or_not"] = temp
  data.to_csv(f"{name}.csv", index=False)

In [22]:
# Predict the outputs and create the final csv file
predict_op()
create_csv("submission_amugalih")